Official U.S. Department of State Visa Appointment Service
Schedule Appointments

https://ais.usvisa-info.com/

This version of the program was written in Feb 2024.

The following functions may not work properly in the future due to the change of website.

But the basic structure of the program can be used.

In this noteboot, all blocks start with "TEST", you can ignore that.

Tested: Chrome✅  Firefox✅
System: Windows✅ MacOS✅


### 1. Install environment
##### pip install selenium==4.17.2
##### pip install tqdm

### 2. Edit User Configs

In [1]:
# User Configs
before_date=20240429 # 在某天之前，格式: YYYYMMDD
dateExcluded=["20240422","20231231"] # 排除日期，格式: YYYYMMDD
maxNumOfNextToBeClicked=4 # 日历最多往后点几下
userEmail="124@qq.com"
userPassword="gerppp&regop-2tA"

# 以下值可以不改动
# Not necessary to change the values down

# timeWait的数值我不确定，可以多尝试几次采用最佳值，或者使用代理
# I'm not sure about the value of timeWait
# timeWaitUntilNextRequest, 单位：秒，每次请求无果后等待时间 , 当值为负数时使用随机时间
# timeWaitUntilNextRequest, Unit: second, If <0, Sleep in random time range=>timeWaitRandomRange
timeWaitUntilNextRequest=-60*5 

# 随机时间范围, 单位：秒
# timeWaitRandomRange, Unit: second
timeWaitRandomRange=(60*10,60*18) 

# 达到请求速率会出现以下问题：The connection has timed out 或者 Site under maintenance
# Error may meet: The connection has timed out / Site under maintenance
# 每次出现问题后等待多少时间重新请求, 单位：秒
# timeWaitWhenErrorCaught, Unit: second
timeWaitWhenErrorCaught=60*60 


# 以下是大使馆在列表中的位置，类表从0开始，如果这个值是负数的话，就不来回切换大使馆位置，直接刷新页面，这样容易被封
# If <-1, Page will be refreshed, Your IP will be blocked easily. 
# If ==-1, Location will be switched automatically to index0 and back to original position.
# When>0, For example in the UK, 0 is empty, 1 is Belfast, 2 is London. Location will be switch to 0 and back to the value.
ConsularSectionLocation=-1

In [2]:
import sys
import time
import json
import random
#import warnings
#import keyboard
from tqdm import tqdm
from pathlib import Path
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.expected_conditions import presence_of_element_located
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.firefox.options import Options as FirefoxOptions
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
driver = webdriver.Firefox()# Browser: FireFox
#driver = webdriver.Chrome()# Browser: Chrome
driver.implicitly_wait(10) # seconds    
driver.maximize_window()
print(driver)

<selenium.webdriver.firefox.webdriver.WebDriver (session="537dd9ca-95cc-47e2-91fd-ba3834630965")>


In [3]:
def random_sleep():
    t = random.randint(5, 10)
    print(f"Waiting For {t} Seconds：", end='', flush=True)
    for i in range(t):
        print('.', end='', flush=True)
        time.sleep(1)
    print('')

def timeout(t):
    try:
        t=int(t)
        for _ in tqdm(range(t), desc="Timeout"):
            time.sleep(1)
        return True
    except Exception as ex:
        print(f"Error in the function timeout, {str(ex)}, sleep for 1s and return False ")
        time.sleep(1)
        return False
        
def smartSleep():
    if timeWaitUntilNextRequest<0:
        random_sleep_time=random.randint(timeWaitRandomRange[0],timeWaitRandomRange[1])
        print(f"Random Sleep: {str(random_sleep_time)}s")
        log(f"Random Sleep: {str(random_sleep_time)}s")
        timeout(random_sleep_time)
    else:
        print(f"timeWaitUntilNextRequest: {str(timeWaitUntilNextRequest)}s")
        log(f"timeWaitUntilNextRequest: {str(timeWaitUntilNextRequest)}s")
        timeout(timeWaitUntilNextRequest)
            
def month_to_number(month_name):  
    month_name=str(month_name)
    month_mapping = {
        "january": 1,
        "february": 2,
        "march": 3,
        "april": 4,
        "may": 5,
        "june": 6,
        "july": 7,
        "august": 8,
        "september": 9,
        "october": 10,
        "november": 11,
        "december": 12,
    }
    month_name_lower = month_name.lower()
    return month_mapping.get(month_name_lower)
def log(content):
    log_content=f"[{str(time.time())}] [{userEmail}] {content}\n"
    with open(f"log/{userEmail}.txt","a+") as f:
        f.write(log_content)

def check_date(date):
    '''
    check if the date given is in valid range and valid format.
    If in range: return True
    If not or caught an error: return False
    '''
    try:
        if len(str(date))!=len(str(before_date)):
            print(f"invalid date length in the function check_date, date given: {str(date)}, compare to date: {str(before_date)}")
            return False
        if 20220101<=int(date)<=20991231==False or 20220101<=int(before_date)<=20991231==False:
            print(f"invalid date range in the function check_date, date given: {str(date)}, compare to date: {str(before_date)}")
            return False
        if int(date)<=before_date:
            print(f"{str(date)} <= {str(before_date)}")
            return True
        else:
            print(f"{str(date)} > {str(before_date)}")
            return False
    except Exception as ex:
        print(f"Error caught in function check_date: {str(ex)}")
        return False
    
def check_date_in_the_excluded_list(date):
    '''
    check if the date is in the excluded list
    If in: return True
    If not or caught an error: return False
    '''
    try:
        for i in dateExcluded:
            if str(date)==i:
                return True
        return False
    except Exception as ex:
        return False
    
print(f"PROGRAM STARTS, target date before: {str(before_date)}")
log(f"PROGRAM STARTS, target date before: {str(before_date)}")
#print(check_date(20240701))
#print(check_date(20240101))

PROGRAM STARTS, target date before: 20240429


In [4]:
# Login Process
def loginProcess(driver):
    driver.get("https://ais.usvisa-info.com/en-gb/niv/users/sign_in")
    random_sleep()
    # Enter userDetails
    driver.find_element(By.NAME, "user\[email\]").send_keys(userEmail)  # Fill Username
    driver.find_element(By.NAME, "user\[password\]").send_keys(userPassword)  # Fill Password
    random_sleep()
    # Click icheckbox
    checkbox_div = driver.find_element(By.CSS_SELECTOR, 'div.icheckbox')
    checkbox_div.click()
    random_sleep()
    # Click commit
    submit_button = driver.find_element(By.NAME, 'commit')
    submit_button.click()
    random_sleep()
    return True

# Goto The Reschedule Appointment Page
def gotoRescheduleAppointmentPage(driver):
    # Press Continue
    continue_link = driver.find_element(By.CSS_SELECTOR, "a.button.primary.small")
    continue_link.click()
    random_sleep()
    # Press Reschedule Appointment
    '''
    # Use XPath, Not necessary
    h5_element = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, "//h5[contains(., 'Reschedule Appointment')]"))
    )
    h5_element.click()
    '''
    link = driver.find_element(By.LINK_TEXT, "Reschedule Appointment")
    link.click()
    random_sleep()
    # Press Reschedule Appointment
    # Find all text "Reschedule Appointment"
    links = driver.find_elements(By.LINK_TEXT, "Reschedule Appointment")
    if len(links) >= 2:
        # Click the second one
        links[1].click()
        return True
    else:
        print("没有找到足够的链接")
        return False


In [5]:
loginProcess(driver)
gotoRescheduleAppointmentPage(driver)

Waiting For 7 Seconds：.......
Waiting For 8 Seconds：........
Waiting For 8 Seconds：........
Waiting For 10 Seconds：..........
Waiting For 10 Seconds：..........
Waiting For 6 Seconds：......


True

In [ ]:
# TEST 1
# Click to open the date selector
date_input = driver.find_element(By.ID, "appointments_consulate_appointment_date")
date_input.click()

In [ ]:
# TEST 2
# Reading datepicker test1
WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.ID, 'ui-datepicker-div')))
datepicker_div = driver.find_element(By.ID, 'ui-datepicker-div')

# 定位到所有的日期分组
groups = datepicker_div.find_elements(By.XPATH, ".//div[contains(@class,'ui-datepicker-group')]")

for group in groups:
    # 确定分组名称
    if 'ui-datepicker-group-first' in group.get_attribute('class'):
        group_name = "First Group"
    elif 'ui-datepicker-group-last' in group.get_attribute('class'):
        group_name = "Last Group"
    else:
        group_name = "Unknown Group"
    
    # 提取分组内的月份和年份
    month_year_element = group.find_element(By.CLASS_NAME, 'ui-datepicker-title')
    month = month_year_element.find_element(By.CLASS_NAME, 'ui-datepicker-month').text
    year = month_year_element.find_element(By.CLASS_NAME, 'ui-datepicker-year').text
    print(f"Group: {group_name}, Month: {month}, Year: {year}")
    
    # 找到分组内所有日期元素
    dates = group.find_elements(By.XPATH, ".//td[@data-handler='selectDay'] | .//td[contains(@class, 'ui-datepicker-unselectable')]")
    
    # 遍历分组内所有日期元素
    for date in dates:
        # 获取日期元素的类名
        class_attribute = date.get_attribute('class')
        # 判断日期是否可选
        is_selectable = 'ui-datepicker-unselectable' not in class_attribute
        # 获取日期文本
        date_text = date.text.strip()  # 使用strip()移除可能的空白字符
        # 如果日期文本不为空，则打印日期和其是否可选
        if date_text:
            print(f"Date: {date_text}, Selectable: {is_selectable}, Belongs to: {group_name}")

In [ ]:
# TEST3

select_day = driver.find_element(By.XPATH, "//td[@data-handler='selectDay' and @data-month='5' and @data-year='2024']/a")
select_day.click()


In [ ]:
# TEST4
# 执行刷新操作
driver.refresh()

In [ ]:
# TEST5

# 定位到<select>元素
select_element = driver.find_element(By.ID, "appointments_consulate_appointment_time")

# 使用Select类来交互<select>元素
select_object = Select(select_element)

# 选择第一个<option>，即索引为1的选项
select_object.select_by_index(1)

#submit_button = driver.find_element(By.ID, "appointments_submit")
#submit_button.click()



In [6]:
def get_selected_option_index(select_object):
    """
    返回当前选中的<option>的索引。

    参数:
    select_object (Select): Selenium的Select对象。

    返回:
    int: 当前选中的<option>的索引。如果没有选中的<option>，返回-1。
    """

    # 点击其它元素，先使date_input失去焦点。如果不失去焦点，日期位置不会被刷新
    select_element = driver.find_element(By.ID, "appointments_consulate_appointment_time")
    select_element.click()
    
    # 获取当前选中的<option>的值
    selected_value = select_object.first_selected_option.get_attribute('value')
    
    # 遍历所有<option>，查找与当前选中的<option>值相匹配的<option>
    for index, option in enumerate(select_object.options):
        if option.get_attribute('value') == selected_value:
            return index
    
    # 如果没有找到匹配的<option>，返回-1
    #return -1
    raise ValueError("In the function get_selected_option_index, 没有找到匹配的<option>")


def refreshData(driver):
    try:
        if ConsularSectionLocation<-1:
            driver.refresh()
            return True
        else:
            # 定位到<select>元素
            select_element = driver.find_element(By.ID, "appointments_consulate_appointment_facility_id")
            # 使用Select类来交互<select>元素
            select_object = Select(select_element)
            selected_index = get_selected_option_index(select_object)
            # 选择第0个<option>，即索引为0的选项
            select_object.select_by_index(0)
            random_sleep()
            # Switch back
            if ConsularSectionLocation==-1:
                select_object.select_by_index(selected_index)
            else:
                select_object.select_by_index(ConsularSectionLocation)
            return True
    except Exception as ex:
        driver.refresh()
        return True

def autoRefreshAndCompare(driver):
    while True:
        try:
            # 定位到输入框并点击
            WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.ID,  "appointments_consulate_appointment_date")))
            date_input = driver.find_element(By.ID, "appointments_consulate_appointment_date")
            date_input.click()
            WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.ID, 'ui-datepicker-div')))
            tries=0
            while tries<maxNumOfNextToBeClicked:
                tries+=1
                res=[]
                # 定位到日期选择器的div
                datepicker_div = driver.find_element(By.ID, 'ui-datepicker-div')

                # 定位到所有的日期分组
                groups = datepicker_div.find_elements(By.XPATH, ".//div[contains(@class,'ui-datepicker-group')]")

                for group in groups:
                    # 确定分组名称
                    if 'ui-datepicker-group-first' in group.get_attribute('class'):
                        group_name = "First Group"
                    elif 'ui-datepicker-group-last' in group.get_attribute('class'):
                        group_name = "Last Group"
                    else:
                        group_name = "Unknown Group"

                    # 提取分组内的月份和年份
                    month_year_element = group.find_element(By.CLASS_NAME, 'ui-datepicker-title')
                    month = month_year_element.find_element(By.CLASS_NAME, 'ui-datepicker-month').text
                    year = month_year_element.find_element(By.CLASS_NAME, 'ui-datepicker-year').text
                    #print(f"Group: {group_name}, Month: {month}, Year: {year}")

                    # 找到分组内所有日期元素
                    dates = group.find_elements(By.XPATH, ".//td[@data-handler='selectDay'] | .//td[contains(@class, 'ui-datepicker-unselectable')]")

                    # 遍历分组内所有日期元素
                    for date in dates:
                        # 获取日期元素的类名
                        class_attribute = date.get_attribute('class')
                        # 判断日期是否可选
                        is_selectable = 'ui-datepicker-unselectable' not in class_attribute
                        # 获取日期文本
                        date_text = date.text.strip()  # 使用strip()移除可能的空白字符
                        # 如果日期文本不为空，则打印日期和其是否可选

                        if date_text and is_selectable:
                            info={"year":year,"month":month_to_number(month),"day":date_text,"status":is_selectable}
                            #print(info)
                            res.append(info)
                if len(res)>0:
                    break
                else:
                    # 使用类名定位到"Next"按钮并点击
                    next_button = driver.find_element(By.CLASS_NAME, "ui-datepicker-next")
                    next_button.click()
                    time.sleep(0.3)
            log(f"res: {str(res)}")
            print(res)
            filted_res=[]
            for i in res:
                date=int(i["year"])*10000+i["month"]*100+int(i["day"])
                #print(date)
                if check_date(date)==True:# In valid range and valid format
                    if check_date_in_the_excluded_list(date)==False:# Not in the excluded list
                        filted_res.append(i)
            log(f"filted_res: {str(filted_res)}")
            if len(filted_res)>0:
                print(filted_res)
                #break
                return filted_res
            else:
                smartSleep()
                # 执行刷新操作，直接执行刷新操作容易被封IP，所以采用切换Consular Section Location来实现刷新
                #driver.refresh()
                refreshData(driver)
        except Exception as ex:
            print(ex)
            try:
                wait.until(EC.presence_of_element_located((By.XPATH, "//*[contains(text(), 'Your session has expired')]")))
                #print("Your session has expired")
                loginProcess(driver)
                #gotoRescheduleAppointmentPage(driver)
            except:
                timeout(timeWaitWhenErrorCaught)
                driver.refresh()
            
def selectDateTimeSubmitAndComfirm(driver, filted_res):
    try:
        select_day = driver.find_element(By.XPATH, f"//td[@data-handler='selectDay' and @data-month='{str(filted_res[0]['month']-1)}' and @data-year='{str(filted_res[0]['year'])}']/a")
        select_day.click()   
        time.sleep(0.3)
        # 定位到<select>元素
        select_element = driver.find_element(By.ID, "appointments_consulate_appointment_time")
        # 使用Select类来交互<select>元素
        select_object = Select(select_element)
        # 选择第一个<option>，即索引为1的选项
        select_object.select_by_index(1)
        # Submit
        submit_button = driver.find_element(By.ID, "appointments_submit")
        submit_button.click()
        # <a class="button alert">Confirm</a>
        wait = WebDriverWait(driver, 10)
        # 方法1：使用CSS选择器
        # confirm_button = driver.find_element(By.CSS_SELECTOR, "a.button.alert")
        # 方法2：使用XPath
        #confirm_button = driver.find_element(By.XPATH, "//a[contains(@class, 'button') and contains(@class, 'alert')]")
        # 设置最大等待时间
        # 使用WebDriverWait和expected_conditions等待"Confirm"按钮变为可点击
        confirm_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//a[contains(text(), 'Confirm')]")))
        # 点击按钮
        date=int(filted_res[0]["year"])*10000+filted_res[0]["month"]*100+int(filted_res[0]["day"])
        if check_date(date)==True:
            confirm_button.click()
        #random_sleep()
        wait = WebDriverWait(driver, 10)
    except Exception as e:
        return [False, str(e)]
    # Final check
    try:
        wait.until(EC.presence_of_element_located((By.XPATH, "//*[contains(text(), 'Your appointment could not be scheduled.')]")))
        #print("Your appointment could not be scheduled")
        return [False, "FailToSchedule"]
    except:
        #print("Suc")
        return [True, "Suc"]


In [ ]:
#refreshData(driver)
while True:
    filted_res=autoRefreshAndCompare(driver)  
    final_res=selectDateTimeSubmitAndComfirm(driver, filted_res)
    print(final_res)
    log(str(final_res))
    if final_res[0]==True:
        break
    smartSleep()


[]
Random Sleep: 628s


Timeout: 100%|███████████████████████████████████████████████████████████████████████| 628/628 [10:29<00:00,  1.00s/it]


Waiting For 5 Seconds：.....
[]
Random Sleep: 738s


Timeout: 100%|███████████████████████████████████████████████████████████████████████| 738/738 [12:19<00:00,  1.00s/it]


Waiting For 8 Seconds：........
[]
Random Sleep: 675s


...

Timeout: 100%|███████████████████████████████████████████████████████████████████████| 683/683 [11:24<00:00,  1.00s/it]


Waiting For 5 Seconds：.....
[]
Random Sleep: 940s


Timeout: 100%|███████████████████████████████████████████████████████████████████████| 940/940 [15:41<00:00,  1.00s/it]


Waiting For 6 Seconds：......
[]
Random Sleep: 720s


Timeout:  22%|███████████████▊                                                       | 160/720 [02:40<09:20,  1.00s/it]

In [ ]:
# TEST6
try:
    wait.until(EC.presence_of_element_located((By.XPATH, "//*[contains(text(), 'Your session has expired')]")))
    print("Your session has expired")
    loginProcess(driver)
    #gotoRescheduleAppointmentPage(driver)
except:
    print("Unkown Error, timeWaitWhenErrorCaught")
    timeout(timeWaitWhenErrorCaught)
    driver.refresh()